In [6]:
import pandas as pd
from IPython.display import display

first_df = pd.read_csv(
    "/home/maliha/Programming/dm/Phishing-Website-Classifier/phishing_complete_dataset.csv",
    nrows=10,
)
preprocessed_df = pd.read_csv(
    "/home/maliha/Programming/dm/Phishing-Website-Classifier/phishing_complete_dataset_preprocessed.csv",
    nrows=10,
)
processed_features_df = pd.read_csv(
    "/home/maliha/Programming/dm/Phishing-Website-Classifier/phishing_complete_features.csv"
)


print("First dataset:")
display(first_df.head())
print("\nPreprocessed dataset:")
display(preprocessed_df.head())
print("\nProcessed features dataset:")
display(processed_features_df.head())
processed_features_df.info()

First dataset:


/tmp/ipykernel_3880/3975393700.py:12: DtypeWarning: Columns (46,47) have mixed types. Specify dtype option on import or set low_memory=False.
  processed_features_df = pd.read_csv(


,rec_id,url,website,result,created_date,webpage_code
0,1,http://intego3.info/EXEL/index.php,1613573972338075.html,1,2021-02-17 20:29:32,"<!DOCTYPE html PUBLIC ""-//W3C//DTD HTML 4.01 T..."
1,2,https://www.mathopenref.com/segment.html,1635698138155948.html,0,2021-10-31 16:35:38,<!DOCTYPE html>\n<html>\n <head>\n <title>\n ...
2,3,https://www.computerhope.com/issues/ch000254.htm,1635699228889266.html,0,2021-10-31 16:53:48,<!DOCTYPE html>\n<!--[if lt IE 7]><html class=...
3,4,https://www.investopedia.com/terms/n/next-elev...,1635750062162701.html,0,2021-11-01 12:31:02,"<!DOCTYPE html>\n<html class=""comp no-js terms..."
4,5,https://jobs.emss.org.uk/lcc.aspx,161356510250721.html,0,2021-02-17 18:01:42,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."



Preprocessed dataset:


,rec_id,url,result,created_date,webpage_code
0,1,http://intego3.info/EXEL/index.php,1,2021-02-17 20:29:32,"<!DOCTYPE html PUBLIC ""-//W3C//DTD HTML 4.01 T..."
1,2,https://www.mathopenref.com/segment.html,0,2021-10-31 16:35:38,<!DOCTYPE html>\n<html>\n <head>\n <title>\n S...
2,3,https://www.computerhope.com/issues/ch000254.htm,0,2021-10-31 16:53:48,"<!DOCTYPE html>\n<html itemscope="""" itemtype=""..."
3,4,https://www.investopedia.com/terms/n/next-elev...,0,2021-11-01 12:31:02,"<!DOCTYPE html>\n<html class=""comp no-js terms..."
4,5,https://jobs.emss.org.uk/lcc.aspx,0,2021-02-17 18:01:42,"<!DOCTYPE html PUBLIC ""-//W3C//DTD XHTML 1.0 T..."



Processed features dataset:


,total_tags,div_count,script_count,iframe_count,form_count,input_count,a_tag_count,img_count,meta_count,max_nesting_depth,...,ratio_official_assets,hotlinking_official_assets,text_length,word_count,phishing_keyword_count,exclamation_count,question_count,special_char_ratio,rec_id,result
0,43,7,1,0,1,4,1,4,1,1,...,0.0,0,575,46,2,0,0,0.116522,1,1
1,100,17,4,0,2,3,7,2,4,1,...,0.0,0,3373,355,0,0,0,0.142010,2,0
2,642,19,8,0,2,2,77,5,19,1,...,0.0,0,11553,1501,2,1,5,0.032978,3,0
3,1635,227,6,1,1,1,150,19,32,1,...,0.0,0,33587,2707,2,3,12,0.150296,4,0
4,819,94,17,1,1,8,58,12,4,1,...,0.0,0,8022,652,1,1,2,0.063201,5,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000 entries, 0 to 79999
Data columns (total 81 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   total_tags                    80000 non-null  int64  
 1   div_count                     80000 non-null  int64  
 2   script_count                  80000 non-null  int64  
 3   iframe_count                  80000 non-null  int64  
 4   form_count                    80000 non-null  int64  
 5   input_count                   80000 non-null  int64  
 6   a_tag_count                   80000 non-null  int64  
 7   img_count                     80000 non-null  int64  
 8   meta_count                    80000 non-null  int64  
 9   max_nesting_depth             80000 non-null  int64  
 10  unclosed_tags                 80000 non-null  int64  
 11  has_doctype                   80000 non-null  int64  
 12  html_length                   80000 non-null  int64  
 13  h

# Preprocess the data

In [ ]:
import re
import warnings

# Ignore all warnings
warnings.filterwarnings("ignore")


def remove_surrogates(text: str) -> str:
    """Remove unpaired surrogate characters to prevent encoding errors"""
    try:
        return text.encode("utf-8", errors="surrogatepass").decode(
            "utf-8", errors="ignore"
        )
    except:
        return re.sub(r"[\ud800-\udfff]", "", text)


def remove_html_comments(text: str) -> str:
    """Remove HTML comments using regex (preserves raw HTML structure)"""
    text = re.sub(r"<!--.*?-->", "", text, flags=re.DOTALL)
    return text


def preprocess_webpage_code(html_code):
    """
    Preprocess webpage_code column only.

    Steps:
    1. Remove surrogates (prevents crashes)
    2. Remove HTML comments (pure noise)
    3. Normalize whitespace (reduces noise)
    """
    if pd.isna(html_code) or html_code == "":
        return ""

    try:
        text = str(html_code)

        # Step 1: Remove surrogates
        text = remove_surrogates(text)

        # Step 2: Remove HTML comments
        text = remove_html_comments(text)

        # Step 3: Normalize whitespace
        text = re.sub(r"\n\s*\n", "\n", text)  # Remove blank lines
        text = re.sub(r"[ \t]+", " ", text)  # Multiple spaces -> single space
        text = text.strip()

        return text

    except Exception as e:
        print(f"Error processing row: {str(e)[:100]}")
        return ""


print("Loading dataset...")
dataset = pd.read_csv(
    "/kaggle/input/phishing-website-webcode-dataset/phishing_complete_dataset.csv",
    sep=",",
    quotechar='"',
)
print(f"Original dataset shape: {dataset.shape}")
print(f"Columns: {dataset.columns.tolist()}")

# Check missing values before
print("\n" + "=" * 60)
print("Missing values BEFORE:")
print("=" * 60)
print(dataset.isnull().sum())

# ========== STEP 1: DROP WEBSITE COLUMN ==========
print("\n" + "=" * 60)
print("STEP 1: Dropping 'website' column...")
print("=" * 60)

if "website" in dataset.columns:
    dataset = dataset.drop("website", axis=1)
    print("✓ 'website' column dropped")
else:
    print("⚠ 'website' column not found in dataset")

print(f"New dataset shape: {dataset.shape}")
print(f"Remaining columns: {dataset.columns.tolist()}")

# ========== STEP 2: REMOVE ROWS WITH MISSING VALUES ==========
print("\n" + "=" * 60)
print("STEP 2: Removing rows with missing values...")
print("=" * 60)

rows_before = len(dataset)
print(f"Rows before removing missing values: {rows_before}")

# Remove rows with ANY missing values
dataset.dropna(inplace=True)

dataset.drop_duplicates(inplace=True)

rows_after = len(dataset)
rows_removed = rows_before - rows_after

print(f"Rows after removing missing values: {rows_after}")
print(f"Rows removed: {rows_removed} ({(rows_removed/rows_before)*100:.2f}%)")

# Verify no missing values remain
print("\nMissing values after removal:")
print(dataset.isnull().sum())

# ========== STEP 3: PREPROCESS WEBPAGE_CODE ONLY ==========
print("\n" + "=" * 60)
print("STEP 3: Preprocessing 'webpage_code' column only...")
print("=" * 60)

# Show before example
print("\nBEFORE preprocessing (first 500 chars):")
print("-" * 60)
sample_raw = dataset["webpage_code"].iloc[0]
print(sample_raw[:500] if pd.notna(sample_raw) else "N/A")

# Apply preprocessing ONLY to webpage_code column
print("\nApplying preprocessing...")
dataset["webpage_code"] = dataset["webpage_code"].apply(preprocess_webpage_code)

# Show after example
print("\nAFTER preprocessing (first 500 chars):")
print("-" * 60)
print(dataset["webpage_code"].iloc[0][:500])

# Statistics
print("\n" + "=" * 60)
print("Statistics:")
print("=" * 60)
print(
    f"Average webpage_code length: {dataset['webpage_code'].str.len().mean():.0f} chars"
)
print(f"Min webpage_code length: {dataset['webpage_code'].str.len().min():.0f} chars")
print(f"Max webpage_code length: {dataset['webpage_code'].str.len().max():.0f} chars")

# ========== STEP 4: SAVE PREPROCESSED DATASET ==========
print("\n" + "=" * 60)
print("STEP 4: Saving preprocessed dataset...")
print("=" * 60)

try:
    dataset.to_csv(
        "/kaggle/working/phishing_complete_dataset_preprocessed.csv",
        index=False,
        encoding="utf-8",
        errors="surrogatepass",
    )
    print("✓ Successfully saved to: phishing_complete_dataset_preprocessed.csv")
except Exception as e:
    print(f"✗ Error saving CSV with surrogatepass: {e}")
    print("\nTrying alternative save method...")

    # Only clean webpage_code if there's an issue
    dataset["webpage_code"] = dataset["webpage_code"].apply(
        lambda x: remove_surrogates(str(x)) if pd.notna(x) else x
    )

    dataset.to_csv(
        "/kaggle/working/phishing_complete_dataset_preprocessed.csv",
        index=False,
        encoding="utf-8",
    )
    print("✓ Saved with alternative method")

# Final verification
print("\n" + "=" * 60)
print("FINAL VERIFICATION:")
print("=" * 60)
print(f"Final shape: {dataset.shape}")
print(f"Final columns: {dataset.columns.tolist()}")
print("\nDataset info:")
print(dataset.info())

print("\n" + "=" * 60)
print("✓ PREPROCESSING COMPLETE!")
print("=" * 60)

# Feature engineering

In [ ]:
import pandas as pd
import re
from urllib.parse import urlparse
from collections import Counter
import numpy as np
import math
from typing import Dict, Optional, List, Tuple
import warnings

warnings.filterwarnings("ignore")


# ============================================================================
# CONFIGURATION & CONSTANTS
# ============================================================================

BRAND_KEYWORDS = {
    "microsoft": ["microsoft.com", "office.com", "outlook.com", "live.com"],
    "google": ["google.com", "gmail.com"],
    "apple": ["apple.com", "icloud.com"],
    "paypal": ["paypal.com"],
    "facebook": ["facebook.com", "fb.com"],
    "amazon": ["amazon.com"],
    "netflix": ["netflix.com"],
    "dropbox": ["dropbox.com"],
    "adobe": ["adobe.com"],
    "linkedin": ["linkedin.com"],
    "twitter": ["twitter.com", "x.com"],
    "instagram": ["instagram.com"],
    "ebay": ["ebay.com"],
    "wells fargo": ["wellsfargo.com"],
    "chase": ["chase.com"],
    "bank of america": ["bankofamerica.com"],
}

FREE_HOSTING_DOMAINS = [
    "000webhost",
    "firebaseapp",
    "formspree",
    "herokuapp",
    "netlify.app",
    "github.io",
    "gitlab.io",
    "weebly.com",
    "wix.com",
    "wordpress.com",
    "blogspot.com",
    "tumblr.com",
    "surge.sh",
    "vercel.app",
    "pages.dev",
    "infinityfree",
    "freehosting",
    "rf.gd",
    "ucoz.",
]

PHISHING_KEYWORDS = [
    "verify",
    "account",
    "suspended",
    "confirm",
    "update",
    "secure",
    "login",
    "password",
    "urgent",
    "immediately",
    "expire",
    "limited",
]


# ============================================================================
# UTILITY FUNCTIONS
# ============================================================================


def safe_divide(numerator: float, denominator: float, default: float = 0.0) -> float:
    """Safely divide two numbers, returning default if denominator is 0"""
    return numerator / denominator if denominator != 0 else default


def extract_domain(url: Optional[str]) -> str:
    """Extract domain from URL safely"""
    if not url:
        return ""
    try:
        return urlparse(url).netloc.lower()
    except:
        return ""


def calculate_shannon_entropy(text: str) -> float:
    """Calculate Shannon entropy (measure of randomness/obfuscation)"""
    if not text:
        return 0.0

    counter = Counter(text)
    length = len(text)

    entropy = 0.0
    for count in counter.values():
        probability = count / length
        entropy -= probability * math.log2(probability)

    return entropy


def extract_all_scripts(html: str) -> str:
    """Extract and concatenate all script content"""
    scripts = re.findall(r"<script[^>]*>(.*?)</script>", html, re.DOTALL | re.I)
    return "\n".join(scripts)


def get_text_content(html: str) -> str:
    """Remove all HTML tags to get plain text"""
    return re.sub(r"<[^>]+>", " ", html)


def extract_structural_features(html: str) -> Dict[str, float]:
    """Extract DOM structure and complexity features"""
    features = {}

    # Tag counts
    features["total_tags"] = len(re.findall(r"<[^>]+>", html))
    features["div_count"] = html.count("<div")
    features["script_count"] = html.count("<script")
    features["iframe_count"] = html.count("<iframe")
    features["form_count"] = html.count("<form")
    features["input_count"] = html.count("<input")
    features["a_tag_count"] = html.count("<a ")
    features["img_count"] = html.count("<img")
    features["meta_count"] = html.count("<meta")

    # Nesting depth (approximation)
    max_depth = 0
    current_depth = 0
    for char in html:
        if char == "<":
            current_depth += 1
            max_depth = max(max_depth, current_depth)
        elif char == ">":
            current_depth = max(0, current_depth - 1)
    features["max_nesting_depth"] = max_depth

    # Malformed HTML
    opening_tags = len(re.findall(r"<(?!/)(?!!)[a-zA-Z][^>]*>", html))
    closing_tags = len(re.findall(r"</[a-zA-Z][^>]*>", html))
    features["unclosed_tags"] = abs(opening_tags - closing_tags)
    features["has_doctype"] = 1 if "<!DOCTYPE" in html.upper() else 0

    # HTML length
    features["html_length"] = len(html)
    features["html_length_log"] = np.log1p(len(html))

    return features


def extract_form_features(html: str) -> Dict[str, float]:
    """Extract features related to forms and inputs"""
    features = {}

    # Input field types
    features["password_field_count"] = len(
        re.findall(r'type\s*=\s*["\']password["\']', html, re.I)
    )
    features["email_field_count"] = len(
        re.findall(r'type\s*=\s*["\']email["\']', html, re.I)
    )
    features["text_input_count"] = len(
        re.findall(r'type\s*=\s*["\']text["\']', html, re.I)
    )
    features["hidden_input_count"] = len(
        re.findall(r'type\s*=\s*["\']hidden["\']', html, re.I)
    )
    features["submit_button_count"] = len(
        re.findall(r'type\s*=\s*["\']submit["\']', html, re.I)
    )

    # Form actions
    form_actions = re.findall(r'action\s*=\s*["\']([^"\']+)["\']', html, re.I)
    features["form_has_external_action"] = 0
    features["form_action_suspicious"] = 0

    for action in form_actions:
        if action.startswith("http"):
            features["form_has_external_action"] = 1
        if ".php" in action.lower():
            features["form_action_suspicious"] = 1

    return features


def extract_script_features(html: str) -> Dict[str, float]:
    """Extract JavaScript and obfuscation features"""
    features = {}

    all_scripts = extract_all_scripts(html)
    features["total_script_length"] = len(all_scripts)

    # Dangerous functions
    features["has_eval"] = 1 if "eval(" in all_scripts else 0
    features["has_unescape"] = 1 if "unescape(" in all_scripts else 0
    features["has_document_write"] = 1 if "document.write" in all_scripts else 0
    features["has_settimeout"] = 1 if "setTimeout" in all_scripts else 0
    features["has_setinterval"] = 1 if "setInterval" in all_scripts else 0

    # Obfuscation patterns
    features["has_unicode_escape"] = 1 if re.search(r"\\u[0-9a-fA-F]{4}", html) else 0
    features["has_hex_escape"] = 1 if re.search(r"\\x[0-9a-fA-F]{2}", html) else 0
    features["has_base64"] = 1 if re.search(r"atob\s*\(", all_scripts) else 0
    features["has_packed_js"] = (
        1 if re.search(r"eval\s*\(\s*function\s*\(", all_scripts) else 0
    )
    features["has_charcodeat"] = 1 if "charCodeAt" in all_scripts else 0
    features["has_fromcharcode"] = 1 if "fromCharCode" in all_scripts else 0

    # Encoding counts
    features["unicode_escape_count"] = len(re.findall(r"\\u[0-9a-fA-F]{4}", html))
    features["url_encoding_count"] = len(re.findall(r"%[0-9a-fA-F]{2}", html))

    # Shannon entropy
    features["js_entropy"] = (
        calculate_shannon_entropy(all_scripts) if all_scripts else 0.0
    )
    features["high_entropy_js"] = 1 if features["js_entropy"] > 5.0 else 0

    return features


def extract_url_features(html: str, page_url: Optional[str] = None) -> Dict[str, float]:
    """Extract features from URLs and links"""
    features = {}

    hrefs = re.findall(r'href\s*=\s*["\']([^"\']+)["\']', html, re.I)

    features["total_links"] = len(hrefs)
    features["external_links"] = 0
    features["suspicious_tld_count"] = 0
    features["ip_in_url"] = 0

    suspicious_tlds = [".tk", ".ml", ".ga", ".cf", ".gq", ".info", ".xyz"]

    for href in hrefs:
        if href.startswith("http"):
            features["external_links"] += 1

            if any(tld in href.lower() for tld in suspicious_tlds):
                features["suspicious_tld_count"] += 1

            if re.search(r"https?://\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", href):
                features["ip_in_url"] = 1

    features["external_resources"] = len(re.findall(r'src\s*=\s*["\']http', html, re.I))

    # URL-text mismatch
    link_patterns = re.findall(
        r'<a[^>]+href\s*=\s*["\']([^"\']+)["\'][^>]*>([^<]*)</a>', html, re.I
    )
    features["url_text_mismatch"] = 0

    for url, text in link_patterns:
        text_clean = text.strip().lower()
        if re.search(r"[a-z0-9-]+\.(com|net|org)", text_clean) and url.startswith(
            "http"
        ):
            if text_clean not in url.lower():
                features["url_text_mismatch"] = 1
                break

    return features


def extract_visual_features(html: str) -> Dict[str, float]:
    """Analyze visual layout patterns"""
    features = {}

    # Image-to-text ratio
    img_tags = re.findall(r"<img[^>]*>", html, re.I)
    total_img_area = 0

    for img in img_tags:
        width = re.search(r'width\s*=\s*["\']?(\d+)', img, re.I)
        height = re.search(r'height\s*=\s*["\']?(\d+)', img, re.I)
        if width and height:
            total_img_area += int(width.group(1)) * int(height.group(1))

    text_length = len(get_text_content(html).strip())
    features["total_image_area"] = total_img_area
    features["image_to_text_ratio"] = safe_divide(total_img_area, text_length)
    features["high_image_to_text"] = 1 if features["image_to_text_ratio"] > 100 else 0

    # Z-index patterns
    z_indices = [int(z) for z in re.findall(r"z-index\s*:\s*(\d+)", html, re.I)]
    features["max_z_index"] = max(z_indices) if z_indices else 0
    features["z_index_range"] = (
        max(z_indices) - min(z_indices) if len(z_indices) > 1 else 0
    )
    features["high_z_index_elements"] = sum(1 for z in z_indices if z > 100)

    # Positioning
    features["absolute_position_count"] = len(
        re.findall(r"position\s*:\s*absolute", html, re.I)
    )
    features["overlay_pattern"] = (
        1
        if (features["absolute_position_count"] > 3 and features["max_z_index"] > 10)
        else 0
    )

    # Hidden elements
    features["font_size_zero"] = len(re.findall(r"font-size\s*:\s*0", html, re.I))
    features["opacity_zero"] = len(re.findall(r"opacity\s*:\s*0", html, re.I))
    features["negative_position"] = len(
        re.findall(r"(left|top)\s*:\s*-\d{3,}px", html, re.I)
    )
    features["display_none"] = len(re.findall(r"display\s*:\s*none", html, re.I))
    features["hidden_text_techniques"] = (
        features["font_size_zero"]
        + features["opacity_zero"]
        + features["negative_position"]
    )

    return features


def extract_behavioral_features(html: str) -> Dict[str, float]:
    """Detect suspicious behavioral patterns"""
    features = {}

    all_scripts = extract_all_scripts(html)

    # Delayed modifications
    features["delayed_password_reveal"] = (
        1
        if re.search(r"setTimeout.*type.*password", all_scripts, re.I | re.DOTALL)
        else 0
    )

    dom_patterns = [
        r"setTimeout.*innerHTML",
        r"setTimeout.*createElement",
        r"setTimeout.*appendChild",
        r"setInterval.*style",
    ]
    features["delayed_dom_modification"] = sum(
        1 for p in dom_patterns if re.search(p, all_scripts, re.I | re.DOTALL)
    )

    # Security blocking
    security_patterns = [
        r"keydown.*preventDefault",
        r"keypress.*preventDefault",
        r"contextmenu.*preventDefault",
        r"F12.*preventDefault",
        r"inspect.*preventDefault",
        r"Ctrl.*U.*preventDefault",
    ]
    features["blocks_security_hotkeys"] = sum(
        1 for p in security_patterns if re.search(p, all_scripts, re.I)
    )

    return features


def extract_identity_features(
    html: str, page_url: Optional[str] = None
) -> Dict[str, float]:
    """Check for identity and brand mismatches"""
    features = {}

    domain = extract_domain(page_url)
    text_content = get_text_content(html).lower()

    # Brand mentions
    mentioned_brands = [
        brand for brand in BRAND_KEYWORDS.keys() if brand in text_content
    ]
    features["brand_mention_count"] = len(mentioned_brands)
    features["domain_brand_mismatch"] = 0

    if domain and mentioned_brands:
        domain_matches = any(
            any(official in domain for official in BRAND_KEYWORDS[brand])
            for brand in mentioned_brands
        )
        features["domain_brand_mismatch"] = 0 if domain_matches else 1

    # Form action analysis
    form_actions = re.findall(r'action\s*=\s*["\']([^"\']+)["\']', html, re.I)
    features["form_action_to_free_host"] = 0
    features["form_action_different_domain"] = 0

    for action in form_actions:
        if any(free_host in action.lower() for free_host in FREE_HOSTING_DOMAINS):
            features["form_action_to_free_host"] = 1

        if action.startswith("http") and domain:
            action_domain = extract_domain(action)
            if action_domain and action_domain != domain:
                features["form_action_different_domain"] = 1

    # Favicon
    features["has_favicon"] = (
        1 if re.search(r'rel\s*=\s*["\'][^"\']*icon[^"\']*["\']', html, re.I) else 0
    )
    favicon_urls = re.findall(
        r'rel\s*=\s*["\'][^"\']*icon[^"\']*["\'][^>]*href\s*=\s*["\']([^"\']+)["\']',
        html,
        re.I,
    )
    features["favicon_external"] = 0
    if favicon_urls and domain:
        features["favicon_external"] = (
            1
            if any(f.startswith("http") and domain not in f for f in favicon_urls)
            else 0
        )

    return features


def extract_semantic_features(
    html: str, page_url: Optional[str] = None
) -> Dict[str, float]:
    """Analyze semantic relationships in the DOM"""
    features = {}

    # Brand-password proximity
    text_with_brands = [
        brand for brand in BRAND_KEYWORDS.keys() if brand in html.lower()
    ]
    features["brands_near_password_field"] = 0

    if text_with_brands:
        password_positions = [
            m.start() for m in re.finditer(r'type\s*=\s*["\']password["\']', html, re.I)
        ]

        for brand in text_with_brands:
            brand_positions = [m.start() for m in re.finditer(brand, html, re.I)]
            for p_pos in password_positions:
                for b_pos in brand_positions:
                    if abs(p_pos - b_pos) < 500:
                        features["brands_near_password_field"] = 1
                        break

    # Title analysis
    title_match = re.search(r"<title[^>]*>([^<]+)</title>", html, re.I)
    features["title_has_brand"] = 0
    features["brand_password_distance"] = 0
    features["brand_password_far"] = 0

    if title_match:
        title_text = title_match.group(1).lower()
        title_has_brand = any(brand in title_text for brand in BRAND_KEYWORDS.keys())
        features["title_has_brand"] = 1 if title_has_brand else 0

        if title_has_brand and re.search(r'type\s*=\s*["\']password["\']', html, re.I):
            title_pos = title_match.start()
            password_positions = [
                m.start()
                for m in re.finditer(r'type\s*=\s*["\']password["\']', html, re.I)
            ]
            min_distance = min(abs(title_pos - p_pos) for p_pos in password_positions)
            features["brand_password_distance"] = min_distance
            features["brand_password_far"] = 1 if min_distance > 2000 else 0

    # External resource analysis
    domain = extract_domain(page_url)
    all_resources = []
    all_resources.extend(re.findall(r'src\s*=\s*["\']([^"\']+)["\']', html, re.I))
    all_resources.extend(
        re.findall(r'href\s*=\s*["\']([^"\']+\.css[^"\']*)["\']', html, re.I)
    )
    all_resources.extend(
        re.findall(r'<script[^>]+src\s*=\s*["\']([^"\']+)["\']', html, re.I)
    )

    external_resources = [r for r in all_resources if r.startswith("http")]

    features["total_resources"] = len(all_resources)
    features["external_resources"] = len(external_resources)
    features["official_brand_resources"] = 0
    features["ratio_official_assets"] = 0
    features["hotlinking_official_assets"] = 0

    if external_resources:
        for resource in external_resources:
            resource_lower = resource.lower()
            for brand, official_domains in BRAND_KEYWORDS.items():
                if any(official in resource_lower for official in official_domains):
                    features["official_brand_resources"] += 1
                    break

        features["ratio_official_assets"] = safe_divide(
            features["official_brand_resources"], len(external_resources)
        )

        if features["ratio_official_assets"] > 0.5 and domain:
            is_official = any(
                any(official in domain for official in domains)
                for domains in BRAND_KEYWORDS.values()
            )
            features["hotlinking_official_assets"] = 0 if is_official else 1

    return features


def extract_content_features(html: str) -> Dict[str, float]:
    """Extract features from visible text and keywords"""
    features = {}

    text = get_text_content(html).lower()

    features["text_length"] = len(text)
    features["word_count"] = len(text.split())
    features["phishing_keyword_count"] = sum(
        1 for keyword in PHISHING_KEYWORDS if keyword in text
    )
    features["exclamation_count"] = text.count("!")
    features["question_count"] = text.count("?")
    features["special_char_ratio"] = safe_divide(
        len(re.findall(r"[^a-zA-Z0-9\s]", text)), len(text)
    )

    return features


def extract_all_features(
    html: str,
    page_url: Optional[str] = None,
    rec_id: Optional[int] = None,
    result: Optional[int] = None,
) -> Dict[str, float]:
    """
    Extract all phishing detection features from HTML.

    Args:
        html: HTML content as string
        page_url: Optional URL of the page
        rec_id: Optional record ID
        result: Optional label (0=legit, 1=phishing)

    Returns:
        Dictionary of all extracted features
    """
    if pd.isna(html) or html == "":
        return {}

    try:
        features = {}

        # Extract all feature categories
        features.update(extract_structural_features(html))
        features.update(extract_form_features(html))
        features.update(extract_script_features(html))
        features.update(extract_url_features(html, page_url))
        features.update(extract_visual_features(html))
        features.update(extract_behavioral_features(html))
        features.update(extract_identity_features(html, page_url))
        features.update(extract_semantic_features(html, page_url))
        features.update(extract_content_features(html))

        # Add metadata
        if rec_id is not None:
            features["rec_id"] = rec_id
        if result is not None:
            features["result"] = result

        return features

    except Exception as e:
        print(f"Error extracting features: {str(e)[:100]}")
        return {}


def process_dataset(dataset: pd.DataFrame, batch_size: int = 1000) -> pd.DataFrame:
    """
    Process entire dataset and extract features.

    Args:
        dataset: DataFrame with columns: webpage_code, url, rec_id, result
        batch_size: Number of rows to process before showing progress

    Returns:
        DataFrame with all extracted features
    """
    print(f"Processing {len(dataset)} rows...")
    features_list = []

    for idx, row in dataset.iterrows():
        if idx % batch_size == 0:
            print(f"Progress: {idx}/{len(dataset)} ({idx/len(dataset)*100:.1f}%)")

        features = extract_all_features(
            html=row.get("webpage_code", ""),
            page_url=row.get("url", None),
            rec_id=row.get("rec_id", None),
            result=row.get("result", None),
        )
        features_list.append(features)

    print(f"Progress: {len(dataset)}/{len(dataset)} (100.0%)")
    return pd.DataFrame(features_list)


def main():
    """Main execution function"""
    print("=" * 60)
    print("PHISHING FEATURE EXTRACTION")
    print("=" * 60)

    # Load dataset
    print("\n1. Loading dataset...")
    dataset = pd.read_csv(
        "/kaggle/input/phishing-complete-dataset-preprocessed/phishing_complete_dataset_preprocessed.csv"
    )
    print(f"   ✓ Loaded: {dataset.shape}")

    # Clean dataset
    print("\n2. Cleaning dataset...")
    original_size = len(dataset)
    dataset.dropna(inplace=True)
    dataset.drop_duplicates(inplace=True)
    print(f"   ✓ Removed {original_size - len(dataset)} rows")
    print(f"   ✓ Final size: {dataset.shape}")

    # Extract features
    print("\n3. Extracting features...")
    features_df = process_dataset(dataset, batch_size=1000)

    print("\n4. Feature extraction complete!")
    print(
        f"   ✓ Total features: {len(features_df.columns) - 2}"
    )  # -2 for rec_id and result

    # Show sample
    print("\n5. Sample features (first 10):")
    print("-" * 60)
    for i, (key, value) in enumerate(features_df.iloc[0].items()):
        if i < 10:
            print(f"   {key}: {value}")

    # Statistics
    print("\n6. Feature statistics:")
    print("-" * 60)
    numeric_cols = features_df.select_dtypes(include=[np.number]).columns
    numeric_cols = [col for col in numeric_cols if col not in ["rec_id", "result"]]
    print(features_df[numeric_cols].describe().round(2))

    # Save
    print("\n7. Saving features...")
    output_path = "/kaggle/working/phishing_complete_features.csv"
    features_df.to_csv(output_path, index=False)
    print(f"   ✓ Saved to: {output_path}")

    print("\n" + "=" * 60)
    print("✓ FEATURE EXTRACTION COMPLETE!")
    print("=" * 60)

    return features_df


if __name__ == "__main__":
    features_df = main()

Progress: 24000/80000 (30.0%)
Progress: 25000/80000 (31.2%)
Progress: 26000/80000 (32.5%)
Progress: 27000/80000 (33.8%)
